In [2]:
# !pip install datasets
!pip install wurlitzer
!pip install ml-dtypes==0.2.0
!pip install keras==2.15.0
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [34]:
import os
import numpy as np
import h5py
import cv2
import tifffile

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from datetime import datetime

import keras
import tensorflow

In [13]:
output_image_dir = "/kaggle/working/images"
output_label_dir = "/kaggle/working/masks"
if not os.path.exists(output_image_dir):
    os.mkdir(output_image_dir)
if not os.path.exists(output_label_dir):
    os.mkdir(output_label_dir)

In [14]:
def list_files(folder, extension='.tiff'):
    filenames = os.listdir(folder)
    filepaths = [os.path.join(folder, filename) for filename in filenames]
    return filepaths

def load_image(file_path):
    tiff_data = tifffile.imread(file_path)
    np_array = np.array(tiff_data)
    return np_array

def binarize(image):
    _, binary = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY)
    return binary

def create_crops(image, label, size=256):
    crops = []
    labels = []
    h, w, _ = image.shape
    for i in range(0, h - size + 1, size):
        for j in range(0, w - size + 1, size):
            crop_img = image[i:i+size, j:j+size]
            crop_label = label[i:i+size, j:j+size]
            if np.mean(crop_label) > 0.01 and 0 < np.mean(crop_img) < 255:  # Checking if label is not 99% black
                crops.append(crop_img)
                labels.append(crop_label)
    return crops, labels

In [15]:
def crop_and_map_labels():
    output_image_dir = "/kaggle/working/images"
    output_label_dir = "/kaggle/working/labels"
    
    if not os.path.exists(output_image_dir):
        os.mkdir(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.mkdir(output_label_dir)

        
    folders = [
        '/kaggle/input/massachusetts-roads-dataset/tiff/train',
        '/kaggle/input/massachusetts-roads-dataset/tiff/test',
        '/kaggle/input/massachusetts-roads-dataset/tiff/val'
    ]
    
    count = 0
    for folder in folders:
        image_folder = f'{folder}/'
        label_folder = f'{folder}_labels/'

        label_paths = list_files(label_folder)
        print(label_paths[0:5])

        for lbl_path in label_paths:
            image_path = os.path.join(image_folder, os.path.basename(lbl_path) + 'f')
            if not os.path.exists(image_path):
                print(f"{image_path} doesn't exist")
                continue
            image = load_image(os.path.join(image_folder, os.path.basename(lbl_path)+'f'))
            label = load_image(lbl_path)

            crops, labels = create_crops(image, label)

            for idx, (crop_img, crop_label) in enumerate(zip(crops, labels)):
                crop_img = crop_img
                crop_label = binarize(crop_label)

                cv2.imwrite(os.path.join(output_image_dir, f"{count}.png"), crop_img)
                cv2.imwrite(os.path.join(output_label_dir, f"{count}.png"), crop_label)
                count = count+1
                


crop_and_map_labels()

['/kaggle/input/massachusetts-roads-dataset/tiff/train_labels/24778855_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/train_labels/24928900_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/train_labels/10828825_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/train_labels/21778960_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/train_labels/25979290_15.tif']
['/kaggle/input/massachusetts-roads-dataset/tiff/test_labels/26578720_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/test_labels/23278930_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/test_labels/10378780_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/test_labels/22078975_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/test_labels/11278840_15.tif']
['/kaggle/input/massachusetts-roads-dataset/tiff/val_labels/10978735_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/val_labels/10228690_15.tif', '/kaggle/input/massachusetts-roads-dataset/tiff/val_labels

In [16]:
paths = {"train":["/kaggle/working/images", "/kaggle/working/labels"]}
    
seed=24
batch_size= 16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
img_data_gen_args = dict(rescale = 1/255.)
mask_data_gen_args = dict()

#If You need to resize images then add this to the flow_from_directory parameters 
#target_size=(150, 150), #Or whatever the size is for your network

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_generator = image_data_generator.flow_from_directory(paths["train"][0], 
                                                           seed=seed, 
                                                           batch_size=batch_size,
                                                           classes = ["."],
                                                           class_mode=None #Very important to set this otherwise it returns multiple numpy arrays thinking class mode is binary.
                                                           )  

mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
mask_generator = mask_data_generator.flow_from_directory(paths["train"][1], 
                                                        classes = ["."],
                                                        seed=seed, 
                                                        batch_size=batch_size,
                                                        color_mode = 'grayscale',   #Read masks in grayscale
                                                         class_mode=None
                                                         )


train_generator = zip(image_generator, mask_generator)


Found 20975 images belonging to 1 classes.
Found 20975 images belonging to 1 classes.


In [36]:
def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    
    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    
    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)
    
    # Decoder
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)
    
    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)
    
    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)
    
    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Dropout(0.1)(c9)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = Model(inputs, outputs, name="U-NET")
    return model


In [37]:
IMAGE_HEIGHT = IMAGE_WIDTH = 256

In [38]:
u_net_model = unet(input_size=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

In [39]:
u_net_model.summary()
# tensorflow.keras.utils.plot_model(u_net_model, show_shapes=True)

Model: "U-NET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 256, 256,  │      1,792 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_36[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 256, 256,  │     36,928 │ dropout_18[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_37[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 128, 128,  │     73,856 │ max_pooling2d_8[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_38[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_19          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 128, 128,  │    147,584 │ dropout_19[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_39[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 64, 64,    │    295,168 │ max_pooling2d_9[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_40[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_20          │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Dropout)           │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 64, 64,    │    590,080 │ dropout_20[0][0]

 Total params: 31,365,313 (119.65 MB)

 Trainable params: 31,353,665 (119.60 MB)

 Non-trainable params: 11,648 (45.50 KB)

In [46]:
def iou_coef(y_true, y_pred, smooth=1e-6):
    intersection = tensorflow.reduce_sum(y_true * y_pred)
    union = tensorflow.reduce_sum(y_true) + tensorflow.reduce_sum(y_pred) - intersection
    
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1e-6):
    intersection = tensorflow.reduce_sum(y_true * y_pred)
    total = tensorflow.reduce_sum(y_true) + tensorflow.reduce_sum(y_pred)
    
    return (2. * intersection + smooth) / (total + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [47]:
EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 16

In [48]:
model_path = "/kaggle/working/aerial-image-road-segmentation-xp.keras"
checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4)

In [49]:
opt = keras.optimizers.Adam(LEARNING_RATE)
u_net_model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])

In [50]:
def create_generator(zipped):
    for (img, mask) in zipped:
        yield (img, mask)

In [ ]:
history = u_net_model.fit(create_generator(train_generator), 
    epochs=EPOCHS,
    steps_per_epoch = len(os.listdir(paths["train"][0])) // BATCH_SIZE,
    batch_size = BATCH_SIZE,
    callbacks = [checkpointer, earlystopper, lr_reducer],
)

Epoch 1/100


2024-07-24 10:49:44.881586: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.11841, expected 2.65729
2024-07-24 10:49:44.881874: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 27392: 3.3488, expected 2.88768
2024-07-24 10:49:44.882008: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 38656: 3.55356, expected 3.09244
2024-07-24 10:49:44.882226: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65536: 3.62512, expected 2.96419
2024-07-24 10:49:44.882251: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65537: 4.47177, expected 3.81084
2024-07-24 10:49:44.882263: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65538: 5.40808, expected 4.74715
2024-07-24 10:49:44.882275: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65543: 5.33073, expected 4.66979
2024-07-24 10:49:44.882286: E ex

1306/1310 ━━━━━━━━━━━━━━━━━━━━ 1s 372ms/step - iou_coef: 0.2460 - loss: 0.6123

2024-07-24 10:59:18.316199: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.43987, expected 2.00293
2024-07-24 10:59:18.316263: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 32: 3.08335, expected 2.64642
2024-07-24 10:59:18.316273: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 179: 3.04435, expected 2.60742
2024-07-24 10:59:18.316291: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2303: 3.03867, expected 2.60173
2024-07-24 10:59:18.316304: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3584: 3.31627, expected 2.87933
2024-07-24 10:59:18.316316: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4607: 2.83112, expected 2.39418
2024-07-24 10:59:18.316326: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5119: 3.27289, expected 2.83596
2024-07-24 10:59:18.316337: E external/l

1310/1310 ━━━━━━━━━━━━━━━━━━━━ 666s 432ms/step - iou_coef: 0.2463 - loss: 0.6120 - learning_rate: 1.0000e-04
Epoch 2/100


/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: iou_coef,loss
  current = self.get_monitor_value(logs)
/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: iou_coef,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


1310/1310 ━━━━━━━━━━━━━━━━━━━━ 488s 372ms/step - iou_coef: 0.4815 - loss: 0.3520 - learning_rate: 1.0000e-04
Epoch 3/100
 544/1310 ━━━━━━━━━━━━━━━━━━━━ 4:45 372ms/step - iou_coef: 0.5452 - loss: 0.2960

In [ ]:
model.save(model_path)

In [ ]:
model = load_model(model_path, custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})

model.evaluate(test_images, test_masks)

predictions = model.predict(test_images, verbose=1)

thresh_val = 0.1
predicton_threshold = (predictions > thresh_val).astype(np.uint8)

ix = random.randint(0, len(predictions))
num_samples = 10

f = plt.figure(figsize = (15, 25))
for i in range(1, num_samples*4, 4):
  ix = random.randint(0, len(predictions))

  f.add_subplot(num_samples, 4, i)
  imshow(test_images[ix][:,:,0])
  plt.title("Image")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+1)
  imshow(np.squeeze(test_masks[ix][:,:,0]))
  plt.title("Groud Truth")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+2)
  imshow(np.squeeze(predictions[ix][:,:,0]))
  plt.title("Prediction")
  plt.axis('off')

  f.add_subplot(num_samples, 4, i+3)
  imshow(np.squeeze(predicton_threshold[ix][:,:,0]))
  plt.title("thresholded at {}".format(thresh_val))
  plt.axis('off')

plt.show()

